In [ ]:
import osmnx as ox
import geopandas as gpd
import pandas as pd
import requests
import time
from shapely.geometry import box

# --- 1. Baixar o polígono do arthur alvim ---
bairro = ox.geocode_to_gdf("Carrão, São Paulo")

# --- 2. Criar a grade (grid) sobre o bairro ---
def criar_grid(bairro, tamanho=0.005):  # Aproximadamente 200m
    minx, miny, maxx, maxy = bairro.total_bounds
    grid = []
    x = minx
    while x < maxx:
        y = miny
        while y < maxy:
            grid.append(box(x, y, x + tamanho, y + tamanho))
            y += tamanho
        x += tamanho
    grid_gdf = gpd.GeoDataFrame(geometry=grid, crs=bairro.crs)
    grid_gdf = gpd.overlay(grid_gdf, bairro, how='intersection')
    return grid_gdf

grid = criar_grid(bairro)


# --- 3. Corrigir projeção para metros, calcular centroide ---
grid = grid.to_crs(epsg=3857)  # Web Mercator
grid["centroide"] = grid.geometry.centroid

# --- 4. Criar colunas separadas de latitude/longitude
centroides = grid["centroide"].to_crs(epsg=4326)  # voltar para lat/lon
grid["centroide_lon"] = centroides.geometry.x
grid["centroide_lat"] = centroides.geometry.y

# --- 6. Definir hospital e API ---
API_KEY = "5b3ce3597851110001cf624824d680a2a89c4dc48933794aaca7b81d"  # Troque aqui pela sua chave
hospital_coords = [-46.5315852, -23.5605378] 

def calcular_distancia_km(origem, destino, api_key):
    url = "https://api.openrouteservice.org/v2/directions/driving-car/json"
    headers = {
        "Authorization": api_key,
        "Content-Type": "application/json"
    }
    body = {
        "coordinates": [origem, destino]
    }
    try:
        response = requests.post(url, json=body, headers=headers)
        if response.status_code == 200:
            dados = response.json()
            distancia_metros = dados['routes'][0]['summary']['distance']
            distancia_km = distancia_metros / 1000
            return round(distancia_km, 2)
        else:
            print("Erro na API:", response.text)
            return None
    except Exception as e:
        print(f"Erro: {e}")
        return None

# --- 7. Calcular a distância para cada grid ---
distancias_km = []
for i, row in grid.iterrows():
    origem = [row["centroide_lon"], row["centroide_lat"]]
    distancia = calcular_distancia_km(origem, hospital_coords, API_KEY)
    distancias_km.append(distancia)
    time.sleep(1)  # evita limite da API gratuita

grid["distancia_km"] = distancias_km
grid.to_csv(f"estimativa_carrao.csv", index=False)

In [5]:
grid.to_csv(f"estimativa_carrao.csv", index=False)

In [2]:
grid

,bbox_west,bbox_south,bbox_east,bbox_north,place_id,osm_type,osm_id,lat,lon,class,...,place_rank,importance,addresstype,name,display_name,geometry,centroide,centroide_lon,centroide_lat,distancia_km
0,-46.558187,-23.570493,-46.517149,-23.534897,8449587,relation,3049439,-23.55153,-46.537791,boundary,...,18,0.381199,city_district,Carrão,"Carrão, São Paulo, Região Imediata de São Paul...","POLYGON ((-5182277.116 -2699335.727, -5182277....",POINT (-5182289.677 -2699351.66),-46.553300,-23.555624,3.76
1,-46.558187,-23.570493,-46.517149,-23.534897,8449587,relation,3049439,-23.55153,-46.537791,boundary,...,18,0.381199,city_district,Carrão,"Carrão, São Paulo, Região Imediata de São Paul...","POLYGON ((-5182277.116 -2698728.545, -5182277....",POINT (-5182324.967 -2699006.544),-46.553617,-23.552782,3.45
2,-46.558187,-23.570493,-46.517149,-23.534897,8449587,relation,3049439,-23.55153,-46.537791,boundary,...,18,0.381199,city_district,Carrão,"Carrão, São Paulo, Região Imediata de São Paul...","POLYGON ((-5182277.116 -2698121.387, -5182277....",POINT (-5182387.07 -2698390.5),-46.554175,-23.547709,3.76
3,-46.558187,-23.570493,-46.517149,-23.534897,8449587,relation,3049439,-23.55153,-46.537791,boundary,...,18,0.381199,city_district,Carrão,"Carrão, São Paulo, Região Imediata de São Paul...","POLYGON ((-5182277.116 -2697514.252, -5182277....",POINT (-5182467.39 -2697794.273),-46.554897,-23.542799,4.57
4,-46.558187,-23.570493,-46.517149,-23.534897,8449587,relation,3049439,-23.55153,-46.537791,boundary,...,18,0.381199,city_district,Carrão,"Carrão, São Paulo, Região Imediata de São Paul...","POLYGON ((-5182277.116 -2697514.252, -5182728....",POINT (-5182520.592 -2697256.955),-46.555375,-23.538373,4.66
5,-46.558187,-23.570493,-46.517149,-23.534897,8449587,relation,3049439,-23.55153,-46.537791,boundary,...,18,0.381199,city_district,Carrão,"Carrão, São Paulo, Região Imediata de São Paul...","MULTIPOLYGON (((-5181721.226 -2699335.727, -51...",POINT (-5182164.043 -2699366.702),-46.552172,-23.555748,3.76
6,-46.558187,-23.570493,-46.517149,-23.534897,8449587,relation,3049439,-23.55153,-46.537791,boundary,...,18,0.381199,city_district,Carrão,"Carrão, São Paulo, Região Imediata de São Paul...","POLYGON ((-5182277.116 -2699335.727, -5182277....",POINT (-5182001.555 -2699010.628),-46.550712,-23.552815,3.17
7,-46.558187,-23.570493,-46.517149,-23.534897,8449587,relation,3049439,-23.55153,-46.537791,boundary,...,18,0.381199,city_district,Carrão,"Carrão, São Paulo, Região Imediata de São Paul...","POLYGON ((-5182277.116 -2698728.545, -5182277....",POINT (-5181998.817 -2698424.966),-46.550687,-23.547993,3.16
8,-46.558187,-23.570493,-46.517149,-23.534897,8449587,relation,3049439,-23.55153,-46.537791,boundary,...,18,0.381199,city_district,Carrão,"Carrão, São Paulo, Região Imediata de São Paul...","POLYGON ((-5182277.116 -2698121.387, -5182277....",POINT (-5181998.817 -2697817.82),-46.550687,-23.542993,4.05
9,-46.558187,-23.570493,-46.517149,-23.534897,8449587,relation,3049439,-23.55153,-46.537791,boundary,...,18,0.381199,city_district,Carrão,"Carrão, São Paulo, Região Imediata de São Paul...","POLYGON ((-5181720.519 -2696907.14, -5181720.5...",POINT (-5181995.218 -2697215.801),-46.550655,-23.538035,5.18


In [4]:
%pip install folium
import folium
from shapely.geometry import mapping

# Converter o bairro para EPSG:4326 (lat/lon) para exibição no mapa
bairro_wgs84 = bairro.to_crs(epsg=4326)
grid_wgs84 = grid.to_crs(epsg=4326)

# Criar mapa centralizado no bairro
centroide_bairro = bairro_wgs84.geometry.centroid.iloc[0]
m = folium.Map(location=[centroide_bairro.y, centroide_bairro.x], zoom_start=14)

# Adicionar polígono do bairro
folium.GeoJson(bairro_wgs84.geometry.__geo_interface__,
               name="Bairro - Arthur alvim",
               style_function=lambda x: {"color": "black", "fillOpacity": 0.1}
).add_to(m)

# Adicionar os grids
for _, row in grid_wgs84.iterrows():
    folium.GeoJson(mapping(row.geometry),
                   style_function=lambda x: {"color": "blue", "weight": 1, "fillOpacity": 0}
    ).add_to(m)

# Adicionar ponto do hospital
folium.Marker(
    location=[-23.5605378, -46.5315852] ,
    popup="Hospital Itaquera",
    icon=folium.Icon(color='red', icon='plus-sign')
).add_to(m)

# Mostrar mapa
m

# [-46.5315852, -23.5605378] 


[notice] A new release of pip is available: 23.2.1 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip
C:\Users\joth1\AppData\Local\Temp\ipykernel_11932\2409307082.py:10: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroide_bairro = bairro_wgs84.geometry.centroid.iloc[0]
